In [ ]:
import logging
from neural_poisson.data.shapenet import ShapeNetCoreDataset, log
from pathlib import Path
import time

log.setLevel(logging.DEBUG)
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
log.addHandler(handler)

def measure_dataset(dataset, N=100):
    start_time = time.time()
    for i in range(N):
        batch = dataset[i]
    end_time = time.time()
    duration = ((end_time - start_time) / N) * 1000
    print(f"Time taken for 100 iterations (GPU): {duration:.6f} ms")

# define the paths
model_id = "1a04e3eab45ca15dd86060f189eb133"
root_dir = Path("/home/borth/2d-gaussian-splatting/")
shapenet_dir = root_dir / "data/ShapeNetCoreTiny/02691156"
shapenet_path = shapenet_dir / model_id / "models/model_normalized.obj"

In [ ]:
s = time.time()
dataset = ShapeNetCoreDataset(
    path=shapenet_path,
    dist=2.0,
    fov=30.0,
    image_size=256,
    segments=6,   # subsample x4
    resolution=0.001,
    num_chunks=100,
    chunk_size=10_000,
    max_empty_points=100_000,
    max_close_points=100_000,
    max_surface_points=100_000,
    vector_field_mode="k_nearest_neighbors"
)
print(f"Time taken for creation: {((time.time() - s)  * 1000):.6f} ms")

In [ ]:
dataset.plot_point_cloud("points_surface")

In [ ]:
dataset.plot_normal_maps(False)

In [ ]:
from neural_poisson.model.neural_poisson import NeuralPoisson
import open3d as o3d

ckpt_path = "/home/borth/2d-gaussian-splatting/logs/2025-02-19/18-03-57/checkpoints/epoch_099.ckpt"
model = NeuralPoisson.load_from_checkpoint(ckpt_path)

In [ ]:
model.isolevel = 0.25
verts, faces = model.to_mesh()
mesh = o3d.geometry.TriangleMesh()
mesh.vertices = o3d.utility.Vector3dVector(verts.cpu().numpy())
mesh.triangles = o3d.utility.Vector3iVector(faces.cpu().numpy())

In [ ]:
mesh.compute_vertex_normals()
o3d.visualization.draw_plotly([mesh])

In [ ]:
import torch
from pytorch3d.structures import Meshes
from neural_poisson.data.prepare import get_depth_camera_space_attributes, rasterize_attributes
import matplotlib.pyplot as plt

# Assuming verts and faces are tensors
# verts: (V, 3) tensor of vertex positions
# faces: (F, 3) tensor of triangle indices

device = "cuda" if torch.cuda.is_available() else "cpu"

# Example: Move tensors to the same device
verts = verts.to(device)
faces = faces.to(device)

# Create a PyTorch3D Mesh
mesh = Meshes(verts=[verts], faces=[faces])
mesh = dataset.mesh

# camera = dataset.cameras[9]
for camera in dataset.cameras:

    attributes = get_depth_camera_space_attributes(mesh=mesh, camera=camera)
    depth_map, mask = rasterize_attributes(
        mesh=mesh,
        camera=camera,
        attributes=attributes,
        image_size=256,
    )
    plt.imshow(depth_map[0].detach().cpu().numpy())

In [ ]:
m1 = dataset.mesh
m2 = dataset.mesh

In [ ]:
mask.shape